### Chain Using LangGraph
Let us see how can we build a simple chain using Langgraph that uses 4 important concepts:

- How to use chat messages as our graph state
- How to use chat models in graph nodes
- How to bind tools to our LLM in chat models
- How to execute the tools call in our graph nodes

In [1]:
from langgraph.graph import StateGraph, START, END
from dataclasses import dataclass
from typing import TypedDict, Literal
from pydantic import BaseModel
import random

from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()

/Users/srinivas/Documents/Others/My_projects/Python/DSAIML/agenticai/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [12]:
# Initialize Groq model using init_chat_model (newer way of setting up models in langchain)
model = init_chat_model(
    model="llama-3.1-8b-instant",  # or any other Groq model
    model_provider="groq",
    temperature=0.7,
)

In [13]:
model.invoke("Hello")

AIMessage(content='Hello. Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 36, 'total_tokens': 54, 'completion_time': 0.022366318, 'completion_tokens_details': None, 'prompt_time': 0.002549253, 'prompt_tokens_details': None, 'queue_time': 0.006554626, 'total_time': 0.024915571}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4dea31877a', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--ccba5caa-6d81-44ca-a954-5a6d67b99352-0', usage_metadata={'input_tokens': 36, 'output_tokens': 18, 'total_tokens': 54})

#### How to use chat messages as our graph state
##### Messages

We can use messages which can be used to capture different roles within a conversation.
LangChain has various message types including HumanMessage, AIMessage, SystemMessage and ToolMessage.
All these Messages are inherited by BaseMessage, there is also AnyMessage we can use.
These represent a message from the user, from chat model, for the chat model to instruct behavior, and from a tool call.

Every message have these important components.

- content - content of the message
- name - Specify the name of author
- response_metadata - optionally, a dict of metadata (e.g., often populated by model provider for AIMessages)

In [14]:
from langchain_core.messages import AIMessage,HumanMessage

messages=[AIMessage(content=f"Please tell me how can I help",name="LLMModel")]
messages.append(HumanMessage(content=f"I want to learn coding",name="Srini"))
messages.append(AIMessage(content=f"Which programming language you want to learn",name="LLMModel"))
messages.append(HumanMessage(content=f"I want to learn python programming language",name="Srini"))

for message in messages:
    message.pretty_print()


================================== Ai Message ==================================
Name: LLMModel

Please tell me how can I help
================================ Human Message =================================
Name: Srini

I want to learn coding
================================== Ai Message ==================================
Name: LLMModel

Which programming language you want to learn
================================ Human Message =================================
Name: Srini

I want to learn python programming language


In [15]:
response = model.invoke(messages)

In [17]:
response.response_metadata

{'token_usage': {'completion_tokens': 480,
  'prompt_tokens': 76,
  'total_tokens': 556,
  'completion_time': 0.597425884,
  'completion_tokens_details': None,
  'prompt_time': 0.004314098,
  'prompt_tokens_details': None,
  'queue_time': 0.005360877,
  'total_time': 0.601739982},
 'model_name': 'llama-3.1-8b-instant',
 'system_fingerprint': 'fp_4dea31877a',
 'service_tier': 'on_demand',
 'finish_reason': 'stop',
 'logprobs': None,
 'model_provider': 'groq'}

In [18]:
response.content

"Python is a great language to learn, it's easy to read and write, and it's widely used in various fields such as:\n\n1. Data Science and Machine Learning\n2. Web Development\n3. Automation\n4. Scientific Computing\n5. Education\n\nHere's a step-by-step guide to get you started with Python:\n\n**Step 1: Install Python**\n\nYou can download the latest version of Python from the official website: <https://www.python.org/downloads/>\n\n**Step 2: Choose a Text Editor or IDE**\n\nYou'll need a text editor or an Integrated Development Environment (IDE) to write and run your Python code. Some popular options include:\n\n* PyCharm (free community edition)\n* Visual Studio Code (free)\n* Sublime Text (free trial, paid version)\n* Atom (free)\n* Spyder (free)\n\n**Step 3: Learn the Basics**\n\nStart with the basics of Python, such as:\n\n* Variables and data types\n* Operators and control structures\n* Functions and modules\n* Object-Oriented Programming (OOP) concepts\n\nYou can find many onlin

### Tools
Tools can be integrated with the LLM models to interact with external systems. External systems can be API's, third party tools.

Whenever a query is asked the model can choose to call the tool and this query is based on the 
natural language input and this will return an output that matches the tool's schema